In [ ]:
import matplotlib.pyplot as plt
import math
import numpy as np
import keras_ocr
import cv2
import os
import skimage.io
from skimage import measure

In [ ]:
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2)/2)
    y_mid = int((y1 + y2)/2)
    return (x_mid, y_mid)
pipeline = keras_ocr.pipeline.Pipeline()

def remove_text(img_path, pipeline):
    # read image
    img = keras_ocr.tools.read(img_path)
    # generate (word, box) tuples 
    prediction_groups = pipeline.recognize([img])
    mask = np.zeros(img.shape[:2], dtype="uint8")
    for box in prediction_groups[0]:
        x0, y0 = box[1][0]
        x1, y1 = box[1][1] 
        x2, y2 = box[1][2]
        x3, y3 = box[1][3] 
        
        x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
        x_mid1, y_mi1 = midpoint(x0, y0, x3, y3)
        
        thickness = int(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ))
        
        cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mi1), 255,    
        thickness)
        img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)
                 
    return(img)

In [ ]:
#create images without words
directory = "images/segmentationImages"
pipeline = keras_ocr.pipeline.Pipeline()
for photo_file in os.listdir(directory):
    if photo_file.startswith("img"):
        path = os.path.join(directory,photo_file,"input.png")
        img = remove_text(path,pipeline)  #returned image is inverted
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) #convert to RGB
        out_path = os.path.join(directory,photo_file,"noWordsOuput.png")
        cv2.imwrite(out_path,img_rgb)

In [ ]:
def get_properties(img_path):
    # Load image
    img = skimage.io.imread(img_path)
    # Convert to binary image
    bwimg = img > skimage.filters.threshold_otsu(img)
    # Compute region properties
    stats = measure.regionprops(bwimg)

    return stats

def annotate_center(img_path, stats, out_path):
    fig, ax = plt.subplots()
    ax.imshow(img_path)
    for obj in stats:
        y, x = obj.centroid
        ax.plot(x, y, 'ro')
    plt.save_fig(out_path)

In [ ]:
#annotate center
directory = "images/segmentationImages"
for photo_file in os.listdir(directory):
    if photo_file.startswith("img"):
        img_path = os.path.join(directory,photo_file,"noWordsOuput.png")
        stats = get_properties(img_path)
        out_path = os.path.join(directory,photo_file,"annotateCenterOuput.png")
        annotate_center(img_path,stats,out_path)